In [25]:
import xgboost as xgb
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
import pandas as pd
import numpy as np
import tqdm, datetime, pickle
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader

import ngboost as ngb

from sklearn import datasets

import plotnine
from plotnine import *

import seaborn as sns

from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

import numpy as np
from hyperopt import hp, tpe, Trials, STATUS_OK, Trials, anneal
from functools import partial
from hyperopt.fmin import fmin

# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Activation, Flatten

In [2]:
data_train = pd.read_csv("preprocessedDataset/pre1.csv")
numerical_category_fewValues = [
    "homeOwnership", 
    "verificationStatus",
    "initialListStatus",
    "applicationType",
    "n11",
    "n12",
]
numerical_category_manyValues = [
    "regionCode",
    "employmentTitle",
    "purpose",
    "postCode",
    "title",
]
date_type = [
    'issueDate',
    'earliesCreditLine',
    'issueDateDT',
    'earliesCreditLineDT',
    'earliesCreditLineYear',
    'earliesCreditLineMonth',
    'issueYear',
    'issueMonth'
]
numerical_serial = [
    "loanAmnt","interestRate","installment","annualIncome","dti","delinquency_2years","ficoRangeLow","ficoRangeHigh","openAcc",
    "pubRec","pubRecBankruptcies","revolBal","revolUtil","totalAcc","n0","n1","n2","n3",
    "n4","n5","n6","n7","n8","n9","n10","n13","n14",
    "term", 
]
object_serial = [
    "grade",
    "subGrade", 
    "employmentLength"
]

In [3]:
features = [f for f in data_train.columns if f not in ['id', 'isDefault', "policyCode", "issueDate", "earliesCreditLine"] and '_outliers' not in f]
y_full = data_train['isDefault']
x_full = data_train[features]

In [4]:
from sklearn.model_selection import train_test_split

X_, X_test, y_, y_test = train_test_split(x_full, y_full, test_size=0.5, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_, y_, test_size=0.6, random_state=0)
del X_, y_

In [18]:
dtrain = xgb.DMatrix(X_train.to_numpy(), label=y_train.to_numpy())
dtest = xgb.DMatrix(X_valid.to_numpy(), label=y_valid.to_numpy())

# 贝叶斯优化

用hyperopt来搞

https://blog.csdn.net/FontThrone/article/details/85045254

In [20]:
# 自定义hyperopt的参数空间
space = {"max_depth": hp.randint("max_depth", 15),
         # "n_estimators": hp.randint("n_estimators", 300),
         'learning_rate': hp.uniform('learning_rate', 1e-3, 5e-1),
         "subsample": hp.randint("subsample", 5),
         "min_child_weight": hp.randint("min_child_weight", 6),
         }

def argsDict_tranform(argsDict, isPrint=False):
    argsDict["max_depth"] = argsDict["max_depth"] # + 5
    # argsDict['n_estimators'] = argsDict['n_estimators'] + 150
    argsDict["learning_rate"] = argsDict["learning_rate"] * 0.02 + 0.05
    argsDict["subsample"] = argsDict["subsample"] * 0.1 + 0.5
    argsDict["min_child_weight"] = argsDict["min_child_weight"] + 1
    if isPrint:
        print(argsDict)
    else:
        pass

    return argsDict

{'max_depth': <hyperopt.pyll.base.Apply object at 0x7f9ad0c1d370>, 'n_estimators': <hyperopt.pyll.base.Apply object at 0x7f9ad0c1dbb0>, 'learning_rate': <hyperopt.pyll.base.Apply object at 0x7f9ad0c1da90>, 'subsample': <hyperopt.pyll.base.Apply object at 0x7f9ad0c1dc10>, 'min_child_weight': <hyperopt.pyll.base.Apply object at 0x7f9ad0c1d580>}


{'max_depth': <hyperopt.pyll.base.Apply at 0x7f9ad0c1d370>,
 'n_estimators': <hyperopt.pyll.base.Apply at 0x7f9ad0c1dbb0>,
 'learning_rate': <hyperopt.pyll.base.Apply at 0x7f9ad0c1da90>,
 'subsample': <hyperopt.pyll.base.Apply at 0x7f9ad0c1dc10>,
 'min_child_weight': <hyperopt.pyll.base.Apply at 0x7f9ad0c1d580>}

In [36]:
def xgboost_factory(argsDict):
    argsDict = argsDict_tranform(argsDict)
    
    params = {
        'booster':'gbtree',
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'max_depth': argsDict['max_depth'],  # 最大深度
        'scale_pos_weight': 3,
        'learning_rate': argsDict['learning_rate'],  # 学习率
        'reg_lambda': 5,
        'reg_alpha': 0, 
        'colsample_bytree': 0.8,
    }
    params['eval_metric'] = ['auc']

    xrf = xgb.train(
        params, dtrain, 
        num_boost_round=10000, evals=[(dtrain, 'train'), (dtest, 'test')], 
        early_stopping_rounds=100, verbose_eval=50
    )

    prediction = xrf.predict(dtest, ntree_limit=xrf.best_ntree_limit)
    
    return roc_auc_score(
                y_valid, 
                prediction
            )

In [40]:
algo = partial(tpe.suggest, n_startup_jobs=1) # partial(anneal.suggest,) # 
best = fmin(
    xgboost_factory, space, 
    algo=algo, max_evals=20, 
    pass_expr_memo_ctrl=None, 
    trials=Trials(), 
    return_argmin=True,
)
## best_params = fmin(factory, space, algo=partial(anneal.suggest,), max_evals=eval_iters, trials=Trials(),return_argmin=True)

[0]	train-auc:0.82633	test-auc:0.66926                                          
[50]	train-auc:0.97366	test-auc:0.71678                                         
[100]	train-auc:0.99063	test-auc:0.71865                                        
[150]	train-auc:0.99499	test-auc:0.71837                                        
[200]	train-auc:0.99723	test-auc:0.71750                                        
[213]	train-auc:0.99746	test-auc:0.71745                                        
  0%|                                    | 0/20 [01:10<?, ?trial/s, best loss=?]

/Users/minkexiu/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:105: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.



[0]	train-auc:0.82633	test-auc:0.66926                                          
[50]	train-auc:0.97409	test-auc:0.71648                                         
[100]	train-auc:0.99108	test-auc:0.71864                                        
[150]	train-auc:0.99443	test-auc:0.71882                                        
[200]	train-auc:0.99671	test-auc:0.71799                                        
[232]	train-auc:0.99764	test-auc:0.71752                                        
  5%|▌         | 1/20 [02:34<23:21, 73.76s/trial, best loss: 0.7187696070131455]

/Users/minkexiu/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:105: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.



[0]	train-auc:0.90303	test-auc:0.64761                                          
[50]	train-auc:0.99894	test-auc:0.71265                                         
[100]	train-auc:0.99912	test-auc:0.71607                                        
[150]	train-auc:0.99912	test-auc:0.71711                                        
[200]	train-auc:0.99912	test-auc:0.71743                                        
[250]	train-auc:0.99912	test-auc:0.71729                                        
[300]	train-auc:0.99912	test-auc:0.71710                                        
[317]	train-auc:0.99912	test-auc:0.71718                                        
 10%|█         | 2/20 [05:22<23:55, 79.76s/trial, best loss: 0.7187696070131455]

...

[0]	train-auc:0.74821	test-auc:0.69329                                          
[50]	train-auc:0.83215	test-auc:0.72215                                         
[100]	train-auc:0.87198	test-auc:0.72464                                        
[150]	train-auc:0.89367	test-auc:0.72509                                        
[200]	train-auc:0.91007	test-auc:0.72471                                        
[238]	train-auc:0.92097	test-auc:0.72439                                        
 90%|███████▏| 18/20 [58:12<08:39, 259.81s/trial, best loss: 0.7172150355741349]

/Users/minkexiu/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:105: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.



[0]	train-auc:0.80680	test-auc:0.67419                                          
[50]	train-auc:0.95023	test-auc:0.71818                                         
[100]	train-auc:0.97462	test-auc:0.72001                                        
[150]	train-auc:0.98394	test-auc:0.71973                                        
[200]	train-auc:0.99027	test-auc:0.71911                                        
[216]	train-auc:0.99105	test-auc:0.71897                                        
 95%|███████▌| 19/20 [59:17<03:17, 197.88s/trial, best loss: 0.7172150355741349]

/Users/minkexiu/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:105: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.



100%|████████| 20/20 [59:18<00:00, 177.95s/trial, best loss: 0.7172150355741349]


In [41]:
best

{'learning_rate': 0.22557954545205516,
 'max_depth': 9,
 'min_child_weight': 3,
 'n_estimators': 176,
 'subsample': 4}

# GridSearch